In [ ]:
BASE_DIR = '/content/drive/MyDrive/sichuan_data'
DATA_DIR = f'{BASE_DIR}/processed'
PATCH_DIR = f'{BASE_DIR}/patches'

In [ ]:
import os
import numpy as np
import pandas as pd
import ee

os.makedirs(PATCH_DIR, exist_ok=True)


In [ ]:
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()

print('Earth Engine initialised.')


In [ ]:
csv_path = os.path.join(DATA_DIR, 'sample_points.csv')
df = pd.read_csv(csv_path)
df.head()


In [ ]:
N = 50
df_small = df.iloc[:N].copy().reset_index(drop=True)
len(df_small)


In [ ]:
def get_s2_patch(lon, lat, size_m=640):
    """
    Fetch a small Sentinel-2 RGB patch around (lon, lat).

    size_m: half-width of the patch in metres (roughly).
            For 10 m resolution, 320 m ~ 32 pixels; 640 m ~ 64 pixels, etc.
    """
    point = ee.Geometry.Point([float(lon), float(lat)])


    s2 = (
        ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(point)
        .filterDate('2020-01-01', '2020-12-31')
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
        .sort('CLOUDY_PIXEL_PERCENTAGE')
        .first()
    )


    if s2 is None:
        return None


    rgb = s2.select(['B4', 'B3', 'B2'])

    region = point.buffer(size_m).bounds()


    patch_dict = rgb.sampleRectangle(region=region, defaultValue=0).getInfo()


    try:
        r = np.array(patch_dict['B4'])
        g = np.array(patch_dict['B3'])
        b = np.array(patch_dict['B2'])
    except KeyError:
        return None

    patch = np.stack([r, g, b], axis=-1)

    return patch


In [ ]:
patch_meta = []

for idx, row in df_small.iterrows():
    lon = row['lon']
    lat = row['lat']
    print(f'[{idx+1}/{len(df_small)}] fetching patch at lon={lon:.3f}, lat={lat:.3f} ...')

    patch = get_s2_patch(lon, lat, size_m=640)  

    if patch is None:
        print('  -> no patch (probably too cloudy), skipping.')
        continue


    patch = patch.astype('float32')

    fname = f'patch_{idx:04d}.npy'
    fpath = os.path.join(PATCH_DIR, fname)
    np.save(fpath, patch)

    patch_meta.append({
        'file': fname,
        'lon': lon,
        'lat': lat,
        'gdp': row.get('gdp', None),
        'ntl': row.get('ntl', None),
    })

len(patch_meta)


In [ ]:
meta_df = pd.DataFrame(patch_meta)
meta_path = os.path.join(PATCH_DIR, 'patch_metadata.csv')
meta_df.to_csv(meta_path, index=False)
meta_path
